In [1]:
from osgeo import gdal, gdal_array, osr, ogr
from collections import Counter, Iterable
from IPython.core.debugger import set_trace

import itertools
import numpy as np
import pdb
import csv
import rasterio
import shapefile
from urllib.request import urlopen

/projectnb/landsat/users/zhangyt/miniconda3/envs/measures/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  


In [2]:
""" the script is used to read the npy files of 164 tiles -- rearrange the changed pixels and 
  sample accroding to the giant table
  
  FF -- decline: 1 growth: 2
  FN -- fire: 3 insect: 4 logging: 5 others: 6
  NF -- regrowth: 7
  NN -- fire: 8 comb: 9-17
  
  *no changes between FF two classes and neither NF
  FN                                                   NN
  3--4: 1     4--3: 4     5--3: 7     6--3: 10         8--(9, 17): 13
  3--5: 2     4--5: 5     5--4: 8     6--4: 11         (9,17)--8: 14
  3--6: 3     4--6: 6     5--6: 9     6--5: 12
    
"""

tile_list_dir = r'/usr3/graduate/zhangyt/Desktop/ABoVE/bash/'
numpy_dir = r'/projectnb/landsat/users/zhangyt/above/post_processing/analysis/conf_mtx/sample_table/npy/'
output_dir = r'/projectnb/landsat/users/zhangyt/above/post_processing/analysis/conf_mtx/sample_table/'

# 1. rearrange the table to 6 big row classes
# 2. random sample
# 3. read the lat/lon to create shapefile
# 4. set up the google sheets api


In [3]:
# 1. rearrange the table to 6 big row classes
# read tile list to loop through to rewrite the numpy file by the class
tile_list = tile_list_dir + 'tiles_all.txt'
with open(tile_list) as f:
    tile = f.read().splitlines() 

for tile_name in tile:
    print(tile_name)
    
    np_file = numpy_dir + tile_name + '_chngpix.npy'
    np_array = np.load(np_file, mmap_mode='r')
    
    # extract the class starting from FN fire (3--4: 1; 3--5: 2; 3--6: 3); FN insect(4--3: 4; 4--5: 5; 4--6: 6)
    # FN logging(5--3: 7; 5--4: 8; 5--6: 9); FN others(6--3: 10; 6--4: 11; 6--5: 12)
    FNfire_array =  np_array[np_array[:, 6] < 4]
    FNinsect_array = np_array[(np_array[:, 6] > 3) & (np_array[:, 6] < 7)]
    FNlogging_array = np_array[(np_array[:, 6] > 6) & (np_array[:, 6] < 10)]
    FNother_array = np_array[(np_array[:, 6] > 9) & (np_array[:, 6] < 13)]
    
    # the NN classes (NN fire: 8--(9, 17): 13; NN other: (9,17)--8: 14)
    NNfire_array =  np_array[np_array[:, 6] == 13]
    NNother_array = np_array[np_array[:, 6] == 14]
    
    #concatenate the class
    if tile_name == 'Bh01v03':
        out_FNfire = FNfire_array
        out_FNinsect = FNinsect_array
        out_FNlogging = FNlogging_array
        out_FNother = FNother_array
        
        out_NNfire = NNfire_array
        out_NNother = NNother_array
    else:
        out_FNfire = np.concatenate((out_FNfire, FNfire_array))
        out_FNinsect = np.concatenate((out_FNinsect, FNinsect_array))
        out_FNlogging = np.concatenate((out_FNlogging, FNlogging_array))
        out_FNother = np.concatenate((out_FNother, FNother_array))
        
        out_NNfire = np.concatenate((out_NNfire, NNfire_array))
        out_NNother = np.concatenate((out_NNother, NNother_array))


Bh01v03
> <ipython-input-3-dffdc006513a>(26)<module>()
     24     set_trace()
     25     #concatenate the class
---> 26     if tile_name == 'Bh01v03':
     27         out_FNfire = FNfire_array
     28         out_FNinsect = FNinsect_array

ipdb> NNfire_array
array([[    5,  2001,  -164, ...,    60,  8729,    13],
       [    6,  2001,  -164, ...,    60,  9531,    13],
       [    7,  2001,  -164, ...,    60,  9548,    13],
       ...,
       [77387,  2013,  -164, ...,    60,  7091,    13],
       [77388,  2013,  -164, ...,    60,  6968,    13],
       [77389,  2013,  -164, ...,    60,  6970,    13]], dtype=int32)
ipdb> q


BdbQuit: 

In [63]:
out_NNfire.shape

(323681856, 7)

In [35]:
# 2. random sample
#sample_FNfire = out_FNfire[np.random.choice(out_FNfire.shape[0], 100, replace=False), 1:7]
#sample_FNinsect = out_FNinsect[np.random.choice(out_FNinsect.shape[0], 50, replace=False), 1:7]
#sample_FNlogging = out_FNlogging[np.random.choice(out_FNlogging.shape[0], 50, replace=False), 1:7]
sample_FNother = out_FNother[np.random.choice(out_FNother.shape[0], 50, replace=False), 1:7]

#sample_NNfire = out_NNfire[np.random.choice(out_NNfire.shape[0], 100, replace=False), 1:7]
#sample_NNother = out_NNother[np.random.choice(out_NNother.shape[0], 30, replace=False), 1:7]

sample_all = np.concatenate((sample_FNfire, sample_FNinsect, sample_FNlogging, sample_FNother, 
                             sample_NNfire, sample_NNother), axis=0)

# sort by year -- all classes
ind = np.argsort(sample_all[:,0])
smp_all = sample_all[ind]

# add new idx for the sample table
insert_col = np.ones((380, 1), dtype=np.int32)
for i in np.arange(0, smp_all.shape[0], dtype=np.int32):
    insert_col[i] = i
    
smp_tab = np.insert(smp_all, [0], insert_col, axis=1)


In [38]:
smp_tab

array([[   0, 1985, -112, ...,   61, 3755,    9],
       [   1, 1986, -115, ...,   54, 4856,    1],
       [   2, 1986, -118, ...,   59,  236,    1],
       ...,
       [ 377, 2012, -117, ...,   56,  985,   12],
       [ 378, 2012, -162, ...,   60, 5350,   13],
       [ 379, 2012, -162, ...,   60, 4784,   13]], dtype=int32)

In [36]:
# 3. read the lat/lon to create shapefile
shp_file = output_dir + 'above_shp'

# funtion to generate a .prj file
def getWKT_PRJ(epsg_code):
    
    # access projection information
    with urlopen("http://spatialreference.org/ref/epsg/{0}/prettywkt/".format(epsg_code)) as wkt:
        # remove spaces between charachters
        remove_spaces = wkt.read().decode('utf-8').replace(" ", "")
        # place all the text on one line
        output = remove_spaces.replace("\n", "")
        return output

# create a point shapefile
distb_shp = shapefile.Writer(shp_file, shapeType=8)

# for every record there must be a corresponding geometry.
distb_shp.autoBalance = 1

# create the field names and data type for each.
distb_shp.field("P_ID", "N")
distb_shp.field("YEAR", "C")
distb_shp.field("TYPE", "C")
#distb_shp.field("AGENT", "C")
#distb_shp.field("NOTE", "C")


#loop through each of the rows and assign the attributes to variables
for i in np.arange(0, smp_all.shape[0], dtype=np.int32):
    dist_id = smp_tab[i, 0]
    dist_year = smp_tab[i, 1]
    dist_type = smp_tab[i, 6]
    
    if dist_type < 13:
        dist_type = 1        # --FN
    else:
        dist_type = 2        # --NN
            
    longitude = smp_tab[i, 2] + smp_tab[i, 3] / 10000
    latitude = smp_tab[i, 4] + smp_tab[i, 5] / 10000
    
    # add geometry
    coordinates = np.column_stack((longitude, latitude))
    distb_shp.multipoint(coordinates)
    
    # add records
    distb_shp.record(dist_id, dist_year, dist_type)
    
    # count the features
    print(i)


# create a projection file
prj_file = output_dir + 'above_shp.prj'
prj = open(prj_file, "w")
epsg = getWKT_PRJ("4326")
prj.write(epsg)
prj.close()

distb_shp.close()


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [39]:
# write attribute to csv for google sheet
tab_name = output_dir + 'attributes_F.csv'
out_tab = np.ones((380, 3), dtype=np.int32) 
for i in np.arange(0, smp_all.shape[0], dtype=np.int32):
    dist_id = smp_tab[i, 0]
    dist_year = smp_tab[i, 1]
    dist_type = smp_tab[i, 6]
    
    """
    
    if dist_type < 13:
        dist_type = 1        # --FN
    else:
        dist_type = 2        # --NN
    """      
    
    # add records
    out_tab[i,] = (dist_id, dist_year, dist_type)
    
np.savetxt(tab_name, out_tab, delimiter=" ", fmt= '%5.0f')


In [24]:
out_tab

array([[   0, 1985,    1],
       [   1, 1986,    1],
       [   2, 1986,    1],
       ...,
       [ 377, 2011,    2],
       [ 378, 2012,    2],
       [ 379, 2012,    1]], dtype=int32)